In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from scipy import stats

# ==========================================
# 1. FUNCIÓN PARA BUSCAR ARCHIVOS CSV
# ==========================================

def find_csv_files(directorio_raiz='.'):
    directorios_con_csv = []
    for raiz, directorios, archivos in os.walk(directorio_raiz):
        archivos_csv = [archivo for archivo in archivos if archivo.lower().endswith('.csv')]
        if archivos_csv:
            directorios_con_csv.append(raiz)
    return directorios_con_csv

In [ ]:

# ==========================================
# 2. CARGAR Y COMBINAR LOS 3 DATASETS
# ==========================================

def cargar_datasets():
    """
    Función para cargar los 3 datasets de paneles fotovoltaicos
    """
    directorios = find_csv_files()
    print("Directorios que contienen archivos CSV:")
    for directorio in directorios:
        print(f"- {directorio}")
    
    # Buscar los archivos específicos
    environment_data = None
    irradiance_data = None
    electrical_data = None
    
    for directorio in directorios:
        # Environment data
        ruta_env = os.path.join(directorio, 'environment_data.csv')
        if os.path.exists(ruta_env):
            environment_data = pd.read_csv(ruta_env)
            print(f"✓ Cargado: {ruta_env}")
        
        # Irradiance data
        ruta_irr = os.path.join(directorio, 'irradiance_data.csv')
        if os.path.exists(ruta_irr):
            irradiance_data = pd.read_csv(ruta_irr)
            print(f"✓ Cargado: {ruta_irr}")
        
        # Electrical data (inversores)
        ruta_elec = os.path.join(directorio, 'electrical_data.csv')
        if os.path.exists(ruta_elec):
            electrical_data = pd.read_csv(ruta_elec)
            print(f"✓ Cargado: {ruta_elec}")
    
    return environment_data, irradiance_data, electrical_data

def seleccionar_variables_inversor_1(electrical_data):
    """
    Seleccionar solo las variables del inversor 1 y limpiar nombres
    """
    # Seleccionar columnas del inversor 1
    columnas_inv1 = ['measured_on']
    for col in electrical_data.columns:
        if 'inv_01_' in col:
            columnas_inv1.append(col)
    
    df_inv1 = electrical_data[columnas_inv1].copy()
    
    # Limpiar nombres de columnas (quitar el sufijo _inv_XXXXX)
    columnas_limpias = {}
    for columna in df_inv1.columns:
        if columna == 'measured_on':
            columnas_limpias[columna] = columna
        elif columna.startswith('inv_01_'):
            # Extraer solo la parte del tipo de medición
            partes = columna.split('_inv_')
            if len(partes) >= 1:
                nuevo_nombre = partes[0].replace('inv_01_', '')
                columnas_limpias[columna] = nuevo_nombre
    
    df_inv1 = df_inv1.rename(columns=columnas_limpias)
    return df_inv1

def combinar_datasets(environment_data, irradiance_data, electrical_data):
    """
    Realizar INNER JOIN de los 3 datasets por measured_on
    """
    # Seleccionar variables del inversor 1
    df_inv1 = seleccionar_variables_inversor_1(electrical_data)
    
    print("Variables seleccionadas del inversor 1:")
    print(df_inv1.columns.tolist())
    
    # Convertir measured_on a datetime en todos los datasets
    environment_data['measured_on'] = pd.to_datetime(environment_data['measured_on'])
    irradiance_data['measured_on'] = pd.to_datetime(irradiance_data['measured_on'])
    df_inv1['measured_on'] = pd.to_datetime(df_inv1['measured_on'])
    
    # Realizar INNER JOIN
    print("\n📊 Realizando INNER JOIN...")
    df_combined = environment_data.merge(irradiance_data, on='measured_on', how='inner')
    df_final = df_combined.merge(df_inv1, on='measured_on', how='inner')
    
    print(f"✓ Dataset final: {len(df_final)} filas, {len(df_final.columns)} columnas")
    print("Columnas finales:", df_final.columns.tolist())
    
    return df_final

In [ ]:


# ==========================================
# 3. PREPROCESAMIENTO Y PROMEDIOS HORARIOS
# ==========================================

def crear_promedios_horarios(df, metodo_agregacion='mean', incluir_estadisticas=True):
    """
        Crear promedios horarios con opciones mejoradas de suavizado
        
        Parameters:
        -----------
        df : DataFrame
            DataFrame con timestamp en 'measured_on'
        metodo_agregacion : str
            'mean', 'median' o 'weighted_mean'
        incluir_estadisticas : bool
        Si incluir estadísticas adicionales (std, min, max)
    """

    print("📊 CREANDO PROMEDIOS HORARIOS MEJORADOS")
    print("="*50)
    
    df = df.copy()
    df = df.set_index('measured_on')
        # Fin de la selección
    
    print(f"✓ Datos originales: {len(df)} registros")
    print(f"✓ Frecuencia original: {pd.infer_freq(df.index) or 'Variable'}")
    print(f"✓ Período: {df.index.min()} a {df.index.max()}")
    
    # ========================================
    # 1. DIFERENTES MÉTODOS DE AGREGACIÓN
    # ========================================
    
    if metodo_agregacion == 'mean':
        # Promedio simple (tu método actual)
        df_hourly = df.groupby(df.index.floor('H')).mean()
        print("✓ Usando: Promedio aritmético")
        
    elif metodo_agregacion == 'median':
        # Mediana (más robusta a outliers)
        df_hourly = df.groupby(df.index.floor('H')).median()
        print("✓ Usando: Mediana (más robusta)")
        
    elif metodo_agregacion == 'weighted_mean':
        # Promedio ponderado (más peso a datos recientes en la hora)
        df_hourly = _promedio_ponderado_horario(df)
        print("✓ Usando: Promedio ponderado temporal")
    
    # ========================================
    # 2. ESTADÍSTICAS ADICIONALES (OPCIONAL)
    # ========================================
    
    if incluir_estadisticas:
        # Calcular desviación estándar horaria (útil para detectar variabilidad)
        df_std = df.groupby(df.index.floor('H')).std()
        df_min = df.groupby(df.index.floor('H')).min()
        df_max = df.groupby(df.index.floor('H')).max()
        df_count = df.groupby(df.index.floor('H')).count()
        
        # Añadir columnas de estadísticas
        for col in df_hourly.columns:
            if col in df_std.columns:
                df_hourly[f'{col}_std'] = df_std[col]
                df_hourly[f'{col}_min'] = df_min[col]
                df_hourly[f'{col}_max'] = df_max[col]
                df_hourly[f'{col}_count'] = df_count[col]
    
    print(f"✓ Datos horarios: {len(df_hourly)} registros")
    
    # ========================================
    # 3. LIMPIEZA Y FILTRADO MEJORADO
    # ========================================
    
    # Identificar horas con pocos datos (menos de la mitad de las observaciones esperadas)
    if incluir_estadisticas:
        observaciones_esperadas = 4  # 4 observaciones por hora (cada 15 min)
        mask_pocos_datos = df_hourly[f'{df_hourly.columns[0]}_count'] < (observaciones_esperadas / 2)
        
        if mask_pocos_datos.sum() > 0:
            print(f"⚠️ {mask_pocos_datos.sum()} horas con pocos datos (< {observaciones_esperadas/2} obs)")
            # Opcional: marcar estas horas como sospechosas
            df_hourly.loc[mask_pocos_datos, 'calidad_datos'] = 'baja'
        else:
            df_hourly['calidad_datos'] = 'alta'
    
    # Manejo inteligente de valores faltantes
    df_hourly = _manejar_valores_faltantes_inteligente(df_hourly)
    
    # ========================================
    # 4. FILTRADO POR OPERACIÓN (MEJORADO)
    # ========================================
    
    df_hourly_filtered = _filtrar_horas_operacion_mejorado(df_hourly)
    
    print(f"✓ Datos finales (solo operación): {len(df_hourly_filtered)} registros")
    print(f"   - Filtradas {len(df_hourly) - len(df_hourly_filtered)} horas sin generación")
    
    # ========================================
    # 5. VALIDACIÓN DE CALIDAD FINAL
    # ========================================
    
    _validar_calidad_suavizado(df, df_hourly_filtered)
    
    return df_hourly_filtered

def _validar_calidad_suavizado(df_original, df_hourly):
    """
    Validar la calidad del suavizado
    """
    print(f"\n📈 VALIDACIÓN DE CALIDAD DEL SUAVIZADO")
    print("="*45)
    
    # Comparar estadísticas básicas
    print(f"✓ Reducción de datos: {len(df_original)} → {len(df_hourly)} "
          f"({len(df_hourly)/len(df_original)*100:.1f}% retenido)")
    
    # Comparar algunas variables clave
    variables_clave = ['ac_power']
    irradiance_col = [col for col in df_hourly.columns if 'irradiance' in col.lower() and '_std' not in col]
    if irradiance_col:
        variables_clave.append(irradiance_col[0])
    
    for var in variables_clave:
        if var in df_original.columns and var in df_hourly.columns:
            orig_mean = df_original[var].mean()
            hourly_mean = df_hourly[var].mean()
            diferencia_pct = abs(orig_mean - hourly_mean) / orig_mean * 100
            
            print(f"   {var}:")
            print(f"     Original: {orig_mean:.2f} | Horario: {hourly_mean:.2f} "
                  f"(diferencia: {diferencia_pct:.1f}%)")
    
    # Verificar continuidad temporal
    gaps_temporales = df_hourly.index.to_series().diff().dt.total_seconds() / 3600
    gaps_grandes = gaps_temporales[gaps_temporales > 1.5]  # Gaps > 1.5 horas
    
    if len(gaps_grandes) > 0:
        print(f"⚠️ {len(gaps_grandes)} gaps temporales > 1.5 horas detectados")
    else:
        print("✓ Continuidad temporal: OK")

def _promedio_ponderado_horario(df):
    """
    Calcular promedio ponderado dando más peso a datos más recientes en cada hora
    """
    df_hourly_list = []
    
    for hora, grupo in df.groupby(df.index.floor('H')):
        if len(grupo) > 1:
            # Crear pesos: más peso a observaciones más tardías en la hora
            pesos = np.linspace(0.5, 1.0, len(grupo))
            pesos = pesos / pesos.sum()  # Normalizar
            
            # Promedio ponderado
            resultado = {}
            for col in grupo.select_dtypes(include=[np.number]).columns:
                resultado[col] = np.average(grupo[col], weights=pesos)
            
            df_hourly_list.append(pd.Series(resultado, name=hora))
        else:
            # Si solo hay una observación, usarla directamente
            df_hourly_list.append(grupo.iloc[0])
    
    return pd.DataFrame(df_hourly_list)

def _filtrar_horas_operacion_mejorado(df_hourly):
    """
    Filtrado mejorado para horas de operación
    """
    print(f"\n🌞 Filtrando horas de operación...")
    
    # Identificar columnas clave
    irradiance_col = [col for col in df_hourly.columns if 'irradiance' in col.lower() and '_std' not in col][0]
    power_col = 'ac_power'
    
    # Criterios múltiples para operación (más robustos)
    criterios_operacion = [
        df_hourly[power_col] > 10,  # Potencia mínima
        df_hourly[irradiance_col] > 50,  # Irradiancia mínima
        df_hourly.index.hour.isin(range(6, 19))  # Solo horas diurnas (6 AM - 7 PM)
    ]
    
    # Combinar criterios (OR lógico)
    mask_operacion = criterios_operacion[0] | criterios_operacion[1]
    mask_operacion = mask_operacion & criterios_operacion[2]  # AND con horas diurnas
    
    df_hourly_filtered = df_hourly[mask_operacion].copy()
    
    print(f"   - Criterio 1 (Potencia > 10W): {criterios_operacion[0].sum()} horas")
    print(f"   - Criterio 2 (Irradiancia > 50): {criterios_operacion[1].sum()} horas")
    print(f"   - Criterio 3 (Horas diurnas): {criterios_operacion[2].sum()} horas")
    print(f"   - Horas operativas finales: {len(df_hourly_filtered)}")
    
    return df_hourly_filtered

def _manejar_valores_faltantes_inteligente(df_hourly):
    """
    Manejo inteligente de valores faltantes para datos horarios
    """
    print(f"\n🔧 Manejando valores faltantes...")
    
    valores_faltantes_antes = df_hourly.isnull().sum().sum()
    
    if valores_faltantes_antes > 0:
        print(f"   - Valores faltantes encontrados: {valores_faltantes_antes}")
        
        # Para gaps pequeños (≤ 3 horas): interpolación lineal
        df_hourly_interpolado = df_hourly.interpolate(method='linear', limit=3)
        
        # Para gaps más grandes: interpolación estacional (considera patrones diarios)
        df_hourly_interpolado = df_hourly_interpolado.interpolate(method='time', limit=6)
        
        # Eliminar filas que aún tienen NaN después de interpolación
        df_hourly_clean = df_hourly_interpolado.dropna()
        
        valores_faltantes_despues = df_hourly_clean.isnull().sum().sum()
        print(f"   - Valores faltantes después de limpieza: {valores_faltantes_despues}")
        print(f"   - Filas eliminadas: {len(df_hourly) - len(df_hourly_clean)}")
        
        return df_hourly_clean
    else:
        print("   - No se encontraron valores faltantes")
        return df_hourly

# ========================================
# FUNCIÓN DE COMPARACIÓN VISUAL
# ========================================

def comparar_suavizado_visual(df_original, df_hourly, variable='ac_power', dias_muestra=1000):
    """
    Comparar visualmente el efecto del suavizado
    """
    # Tomar una muestra de pocos días para ver el efecto
    fecha_inicio = df_original.index.min()
    fecha_fin = fecha_inicio + pd.Timedelta(days=dias_muestra)
    
    df_muestra = df_original[df_original.index <= fecha_fin]
    df_hourly_muestra = df_hourly[df_hourly.index <= fecha_fin]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # Gráfica 1: Datos originales (cada 15 min)
    ax1.plot(df_muestra.index, df_muestra[variable], alpha=0.7, color='lightblue', 
             linewidth=0.8, label=f'{variable} (cada 15 min)')
    ax1.set_title(f'Datos Originales - {variable} (Muestra de {dias_muestra} días)')
    ax1.set_ylabel(variable)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Gráfica 2: Datos suavizados (cada hora)
    ax2.plot(df_hourly_muestra.index, df_hourly_muestra[variable], alpha=0.8, color='blue', 
             linewidth=2, marker='o', markersize=4, label=f'{variable} (horario)')
    ax2.set_title(f'Datos Suavizados - {variable} (Promedios Horarios)')
    ax2.set_ylabel(variable)
    ax2.set_xlabel('Fecha y Hora')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Formatear fechas
    for ax in [ax1, ax2]:
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 COMPARACIÓN DE SUAVIZADO ({dias_muestra} días):")
    print(f"   - Puntos originales: {len(df_muestra)}")
    print(f"   - Puntos suavizados: {len(df_hourly_muestra)}")
    print(f"   - Factor de reducción: {len(df_muestra)/len(df_hourly_muestra):.1f}x")


# ==========================================
# 4. DETECTORES DE ANOMALÍAS
# ==========================================

def detector_euclidean_distance(df, percentil=99):
    """
    Detector de anomalías con Distancia Euclidiana (1% más lejanos para ser más selectivo)
    """
    print("\n🔍 Detector 1: Distancia Euclidiana")
    
    columnas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
    datos = df[columnas_numericas].to_numpy()
    
    # Normalizar datos (muy importante para evitar que una variable domine)
    scaler = StandardScaler()
    datos_normalizados = scaler.fit_transform(datos)
    
    # Calcular distancias euclidianas al centroide
    centroide = np.mean(datos_normalizados, axis=0)
    distancias = np.sqrt(np.sum((datos_normalizados - centroide)**2, axis=1))
    
    # Umbral más estricto: percentil 99 (1% más lejano)
    umbral = np.percentile(distancias, percentil)
    anomalias_mask = distancias > umbral
    
    anomalias_euclidean = df.index[anomalias_mask]
    
    print(f"   - Umbral: {umbral:.4f}")
    print(f"   - Anomalías detectadas: {len(anomalias_euclidean)} ({len(anomalias_euclidean)/len(df)*100:.2f}%)")
    
    return anomalias_euclidean, distancias, umbral

def detector_mahalanobis_distance(df, percentil=99):
    """
    Detector de anomalías con Distancia Mahalanobis (99% percentil)
    """
    print("\n🔍 Detector 2: Distancia Mahalanobis")
    
    columnas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
    datos = df[columnas_numericas].to_numpy()
    
    # Normalizar datos primero
    scaler = StandardScaler()
    datos_normalizados = scaler.fit_transform(datos)
    
    # Calcular matriz de covarianza y su inversa
    cov_matrix = np.cov(datos_normalizados, rowvar=False)
    
    # Usar pseudoinversa para evitar problemas de singularidad
    try:
        inv_cov_matrix = np.linalg.inv(cov_matrix)
    except:
        inv_cov_matrix = np.linalg.pinv(cov_matrix)
    
    # Calcular distancias de Mahalanobis
    centroide = np.mean(datos_normalizados, axis=0)
    distancias = []
    
    for i in range(len(datos_normalizados)):
        diff = datos_normalizados[i] - centroide
        mahal_dist = np.sqrt(diff.T @ inv_cov_matrix @ diff)
        distancias.append(mahal_dist)
    
    distancias = np.array(distancias)
    
    # Umbral para el percentil especificado
    umbral = np.percentile(distancias, percentil)
    anomalias_mask = distancias > umbral
    
    anomalias_mahalanobis = df.index[anomalias_mask]
    
    print(f"   - Umbral: {umbral:.4f}")
    print(f"   - Anomalías detectadas: {len(anomalias_mahalanobis)} ({len(anomalias_mahalanobis)/len(df)*100:.2f}%)")
    
    return anomalias_mahalanobis, distancias, umbral

def detector_isolation_forest(df, contaminacion=0.01):
    """
    Detector de anomalías con Isolation Forest (1% contaminación más selectivo)
    """
    print("\n🔍 Detector 3: Isolation Forest")
    
    columnas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
    datos = df[columnas_numericas]
    
    # Normalizar datos
    scaler = StandardScaler()
    datos_normalizados = scaler.fit_transform(datos)
    
    # Isolation Forest
    iso_forest = IsolationForest(contamination=contaminacion, random_state=42, n_estimators=200)
    anomalias_pred = iso_forest.fit_predict(datos_normalizados)
    
    # Obtener scores de anomalía
    anomaly_scores = iso_forest.decision_function(datos_normalizados)
    
    # Identificar anomalías (predicción = -1)
    anomalias_mask = anomalias_pred == -1
    anomalias_isolation = df.index[anomalias_mask]
    
    print(f"   - Contaminación: {contaminacion*100}%")
    print(f"   - Anomalías detectadas: {len(anomalias_isolation)} ({len(anomalias_isolation)/len(df)*100:.2f}%)")
    
    return anomalias_isolation, anomaly_scores

# ==========================================
# 5. VISUALIZACIÓN MEJORADA
# ==========================================

def plot_euclidean_distance_with_mask(df, anomalias_euclidean, distancias, umbral, 
                                      variable_principal='ac_voltage'):
    """
    Graficar la distancia euclidiana con la máscara de anomalías
    """
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    # Verificar que la variable principal existe
    if variable_principal not in df.columns:
        posibles = [col for col in df.columns if 'voltage' in col.lower() or 'power' in col.lower()]
        if posibles:
            variable_principal = posibles[0]
        else:
            variable_principal = df.select_dtypes(include=[np.number]).columns[0]
    
    print(f"📊 Graficando variable: {variable_principal}")
    
    # ========================================
    # GRÁFICA 1: DISTANCIAS EUCLIDIANAS
    # ========================================
    
    # Graficar todas las distancias
    axes[0].plot(df.index, distancias, color='blue', alpha=0.7, linewidth=1, 
                label=f'Distancia Euclidiana')
    
    # Línea del umbral
    axes[0].axhline(y=umbral, color='red', linestyle='--', linewidth=2, 
                   label=f'Umbral (percentil 99): {umbral:.4f}')
    
    # Resaltar anomalías detectadas
    mask_anomalias = distancias > umbral
    anomalias_indices = np.where(mask_anomalias)[0]
    
    if len(anomalias_indices) > 0:
        axes[0].scatter(df.index[anomalias_indices], distancias[anomalias_indices], 
                       color='red', s=50, alpha=0.8, zorder=5,
                       label=f'Anomalías detectadas ({len(anomalias_indices)})')
    
    # Rellenar área de anomalías
    axes[0].fill_between(df.index, distancias, umbral, 
                        where=(distancias > umbral), 
                        color='red', alpha=0.2, interpolate=True,
                        label='Zona anómala')
    
    axes[0].set_title('Distancias Euclidianas y Detección de Anomalías')
    axes[0].set_ylabel('Distancia Euclidiana')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # ========================================
    # GRÁFICA 2: VARIABLE PRINCIPAL CON ANOMALÍAS
    # ========================================
    
    # Graficar la variable principal
    axes[1].plot(df.index, df[variable_principal], alpha=0.7, color='lightblue', 
                label=f'{variable_principal} (Normal)')
    
    # Superponer anomalías detectadas
    if len(anomalias_euclidean) > 0:
        axes[1].scatter(anomalias_euclidean, df.loc[anomalias_euclidean, variable_principal], 
                       color='red', s=50, alpha=0.8, zorder=5,
                       label=f'Anomalías ({len(anomalias_euclidean)})')
    
    axes[1].set_title(f'Variable {variable_principal} con Anomalías Detectadas')
    axes[1].set_ylabel(variable_principal)
    axes[1].set_xlabel('Fecha')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Formatear fechas en ambos ejes X
    for ax in axes:
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()
    
    # ========================================
    # ESTADÍSTICAS ADICIONALES
    # ========================================
    print(f"\n📊 ESTADÍSTICAS DE DISTANCIA EUCLIDIANA:")
    print(f"   - Distancia mínima: {np.min(distancias):.4f}")
    print(f"   - Distancia máxima: {np.max(distancias):.4f}")
    print(f"   - Distancia promedio: {np.mean(distancias):.4f}")
    print(f"   - Desviación estándar: {np.std(distancias):.4f}")
    print(f"   - Umbral (percentil 99): {umbral:.4f}")
    print(f"   - Anomalías detectadas: {len(anomalias_euclidean)} ({len(anomalias_euclidean)/len(df)*100:.2f}%)")

def plot_anomalies_comparison_mejorado(df, anomalias_euclidean, anomalias_mahalanobis, anomalias_isolation,
                                      dist_euclidean, dist_mahalanobis, scores_isolation,
                                      umbral_euclidean, umbral_mahalanobis,
                                      variable_principal='ac_voltage'):
    """
    Versión mejorada que incluye las distancias para el método euclidiano
    """
    fig, axes = plt.subplots(4, 1, figsize=(15, 16))
    
    if variable_principal not in df.columns:
        posibles = [col for col in df.columns if 'voltage' in col.lower() or 'power' in col.lower()]
        if posibles:
            variable_principal = posibles[0]
        else:
            variable_principal = df.select_dtypes(include=[np.number]).columns[0]
    
    print(f"📊 Graficando variable: {variable_principal}")
    
    # ========================================
    # GRÁFICA 1: DISTANCIAS EUCLIDIANAS (NUEVA)
    # ========================================
    axes[0].plot(df.index, dist_euclidean, color='blue', alpha=0.7, linewidth=1, 
                label='Distancia Euclidiana')
    axes[0].axhline(y=umbral_euclidean, color='red', linestyle='--', linewidth=2, 
                   label=f'Umbral: {umbral_euclidean:.4f}')
    
    # Resaltar anomalías
    mask_anomalias = dist_euclidean > umbral_euclidean
    anomalias_indices = np.where(mask_anomalias)[0]
    
    if len(anomalias_indices) > 0:
        axes[0].scatter(df.index[anomalias_indices], dist_euclidean[anomalias_indices], 
                       color='red', s=30, alpha=0.8, zorder=5)
        axes[0].fill_between(df.index, dist_euclidean, umbral_euclidean, 
                            where=(dist_euclidean > umbral_euclidean), 
                            color='red', alpha=0.2, interpolate=True)
    
    axes[0].set_title('Distancias Euclidianas y Máscara de Anomalías')
    axes[0].set_ylabel('Distancia Euclidiana')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # ========================================
    # GRÁFICA 2: VARIABLE PRINCIPAL - EUCLIDIANA
    # ========================================
    axes[1].plot(df.index, df[variable_principal], alpha=0.7, color='lightblue', 
                label=variable_principal)
    if len(anomalias_euclidean) > 0:
        axes[1].scatter(anomalias_euclidean, df.loc[anomalias_euclidean, variable_principal], 
                       color='red', s=50, alpha=0.8, label=f'Anomalías Euclidiana ({len(anomalias_euclidean)})')
    axes[1].set_title('Detector 1: Distancia Euclidiana en Variable Principal')
    axes[1].set_ylabel(variable_principal)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # ========================================
    # GRÁFICA 3: DISTANCIA MAHALANOBIS
    # ========================================
    axes[2].plot(df.index, df[variable_principal], alpha=0.7, color='lightblue', 
                label=variable_principal)
    if len(anomalias_mahalanobis) > 0:
        axes[2].scatter(anomalias_mahalanobis, df.loc[anomalias_mahalanobis, variable_principal], 
                       color='orange', s=50, alpha=0.8, label=f'Anomalías Mahalanobis ({len(anomalias_mahalanobis)})')
    axes[2].set_title('Detector 2: Distancia Mahalanobis')
    axes[2].set_ylabel(variable_principal)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    # ========================================
    # GRÁFICA 4: ISOLATION FOREST
    # ========================================
    axes[3].plot(df.index, df[variable_principal], alpha=0.7, color='lightblue', 
                label=variable_principal)
    if len(anomalias_isolation) > 0:
        axes[3].scatter(anomalias_isolation, df.loc[anomalias_isolation, variable_principal], 
                       color='purple', s=50, alpha=0.8, label=f'Anomalías Isolation Forest ({len(anomalias_isolation)})')
    axes[3].set_title('Detector 3: Isolation Forest')
    axes[3].set_ylabel(variable_principal)
    axes[3].set_xlabel('Fecha')
    axes[3].legend()
    axes[3].grid(True, alpha=0.3)
    
    # Formatear fechas en todos los ejes X
    for ax in axes:
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()

def plot_anomalies_comparison(df, anomalias_euclidean, anomalias_mahalanobis, anomalias_isolation, 
                            variable_principal='ac_voltage'):
    """
    Crear gráficas comparativas de los 3 métodos de detección (función original)
    """
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    if variable_principal not in df.columns:
        # Buscar una variable similar
        posibles = [col for col in df.columns if 'voltage' in col.lower() or 'power' in col.lower()]
        if posibles:
            variable_principal = posibles[0]
        else:
            variable_principal = df.select_dtypes(include=[np.number]).columns[0]
    
    print(f"📊 Graficando variable: {variable_principal}")
    
    # Gráfica 1: Distancia Euclidiana
    axes[0].plot(df.index, df[variable_principal], alpha=0.7, label=variable_principal)
    if len(anomalias_euclidean) > 0:
        axes[0].scatter(anomalias_euclidean, df.loc[anomalias_euclidean, variable_principal], 
                       color='red', s=50, alpha=0.8, label=f'Anomalías Euclidiana ({len(anomalias_euclidean)})')
    axes[0].set_title('Detector 1: Distancia Euclidiana')
    axes[0].set_ylabel(variable_principal)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Gráfica 2: Distancia Mahalanobis
    axes[1].plot(df.index, df[variable_principal], alpha=0.7, label=variable_principal)
    if len(anomalias_mahalanobis) > 0:
        axes[1].scatter(anomalias_mahalanobis, df.loc[anomalias_mahalanobis, variable_principal], 
                       color='orange', s=50, alpha=0.8, label=f'Anomalías Mahalanobis ({len(anomalias_mahalanobis)})')
    axes[1].set_title('Detector 2: Distancia Mahalanobis')
    axes[1].set_ylabel(variable_principal)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Gráfica 3: Isolation Forest
    axes[2].plot(df.index, df[variable_principal], alpha=0.7, label=variable_principal)
    if len(anomalias_isolation) > 0:
        axes[2].scatter(anomalias_isolation, df.loc[anomalias_isolation, variable_principal], 
                       color='purple', s=50, alpha=0.8, label=f'Anomalías Isolation Forest ({len(anomalias_isolation)})')
    axes[2].set_title('Detector 3: Isolation Forest')
    axes[2].set_ylabel(variable_principal)
    axes[2].set_xlabel('Fecha')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    # Formatear fechas en el eje X
    for ax in axes:
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()

# ==========================================
# 6. ANÁLISIS Y COMPARACIÓN
# ==========================================

def analizar_calidad_anomalias(df, anomalias_euclidean, anomalias_mahalanobis, anomalias_isolation):
    """
    Analizar la calidad y características de las anomalías detectadas
    """
    print("\n🔬 ANÁLISIS DE CALIDAD DE ANOMALÍAS:")
    print("="*50)
    
    # Estadísticas de las variables en anomalías vs normal
    irradiance_col = [col for col in df.columns if 'irradiance' in col.lower()][0]
    temp_col = [col for col in df.columns if 'temperature' in col.lower()][0]
    
    print("\n📊 Estadísticas comparativas:")
    
    for nombre, anomalias in [("Euclidiana", anomalias_euclidean), 
                              ("Mahalanobis", anomalias_mahalanobis), 
                              ("Isolation Forest", anomalias_isolation)]:
        
        if len(anomalias) > 0:
            print(f"\n{nombre}:")
            
            # Datos normales vs anómalos
            datos_normales = df.drop(anomalias)
            datos_anomalos = df.loc[anomalias]
            
            print(f"  🌞 Irradiancia promedio:")
            print(f"     Normal: {datos_normales[irradiance_col].mean():.2f} W/m²")
            print(f"     Anómalo: {datos_anomalos[irradiance_col].mean():.2f} W/m²")
            
            print(f"  ⚡ AC Power promedio:")
            print(f"     Normal: {datos_normales['ac_power'].mean():.2f} W")
            print(f"     Anómalo: {datos_anomalos['ac_power'].mean():.2f} W")
            
            print(f"  🌡️ Temperatura promedio:")
            print(f"     Normal: {datos_normales[temp_col].mean():.2f} °C")
            print(f"     Anómalo: {datos_anomalos[temp_col].mean():.2f} °C")
            
            # Análisis temporal
            anomalias_por_hora = datos_anomalos.index.hour.value_counts().sort_index()
            hora_mas_anomalias = anomalias_por_hora.idxmax() if len(anomalias_por_hora) > 0 else "N/A"
            print(f"  🕐 Hora con más anomalías: {hora_mas_anomalias}:00")

    


In [ ]:
def generar_lista_anomalias(anomalias_euclidean, anomalias_mahalanobis, anomalias_isolation):
    """
    Generar lista con fechas de anomalías detectadas
    """
    print("\n📋 RESUMEN DE ANOMALÍAS DETECTADAS:")
    print("="*50)
    
    print(f"\n1️⃣ DISTANCIA EUCLIDIANA ({len(anomalias_euclidean)} anomalías):")
    for fecha in sorted(anomalias_euclidean[:10]):  # Solo primeras 10 para no saturar
        print(f"   - {fecha.strftime('%Y-%m-%d %H:%M:%S')}")
    if len(anomalias_euclidean) > 10:
        print(f"   ... y {len(anomalias_euclidean) - 10} más")
    print(f"\n2️⃣ DISTANCIA MAHALANOBIS ({len(anomalias_mahalanobis)} anomalías):")
    for fecha in sorted(anomalias_mahalanobis[:10]):  # Solo primeras 10
        print(f"   - {fecha.strftime('%Y-%m-%d %H:%M:%S')}")
    if len(anomalias_mahalanobis) > 10:
        print(f"   ... y {len(anomalias_mahalanobis) - 10} más")
    
    print(f"\n3️⃣ ISOLATION FOREST ({len(anomalias_isolation)} anomalías):")
    for fecha in sorted(anomalias_isolation[:10]):  # Solo primeras 10
        print(f"   - {fecha.strftime('%Y-%m-%d %H:%M:%S')}")
    if len(anomalias_isolation) > 10:
        print(f"   ... y {len(anomalias_isolation) - 10} más")
    
    # Encontrar anomalías comunes
    anomalias_comunes = set(anomalias_euclidean) & set(anomalias_mahalanobis) & set(anomalias_isolation)
    print(f"\n🎯 ANOMALÍAS DETECTADAS POR LOS 3 MÉTODOS ({len(anomalias_comunes)}):")
    for fecha in sorted(anomalias_comunes):
        print(f"   - {fecha.strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Anomalías únicas de cada método
    solo_euclidean = set(anomalias_euclidean) - set(anomalias_mahalanobis) - set(anomalias_isolation)
    solo_mahalanobis = set(anomalias_mahalanobis) - set(anomalias_euclidean) - set(anomalias_isolation)
    solo_isolation = set(anomalias_isolation) - set(anomalias_euclidean) - set(anomalias_mahalanobis)
    
    print(f"\n🔸 Solo Euclidiana: {len(solo_euclidean)}")
    print(f"🔸 Solo Mahalanobis: {len(solo_mahalanobis)}")
    print(f"🔸 Solo Isolation Forest: {len(solo_isolation)}")

In [ ]:
# ==========================================
# 7. MÉTRICAS DE SEVERIDAD DE ANOMALÍAS
# ==========================================

class SeveridadAnomalias:
    """
    Sistema de métricas para cuantificar la severidad de anomalías
    en paneles fotovoltaicos
    """
    
    def __init__(self):
        self.scaler = StandardScaler()
        self.metricas_calculadas = False
    
    def calcular_metricas_severidad(self, df, anomalias_euclidean, anomalias_mahalanobis, 
                                  anomalias_isolation, dist_euclidean, dist_mahalanobis, 
                                  scores_isolation):
        """
        Calcular múltiples métricas de severidad para cada anomalía
        """
        print("🔍 CALCULANDO MÉTRICAS DE SEVERIDAD")
        print("="*50)
        
        # Variables clave para paneles solares
        irradiance_col = [col for col in df.columns if 'irradiance' in col.lower()][0]
        temp_col = [col for col in df.columns if 'temperature' in col.lower()][0]
        
        # DataFrame para almacenar todas las métricas
        metricas_df = pd.DataFrame(index=df.index)
        
        # ========================================
        # 1. MÉTRICA DE DISTANCIA NORMALIZADA
        # ========================================
        
        # Normalizar distancias euclidianas (0-100)
        dist_euclidean_norm = pd.Series(dist_euclidean, index=df.index)
        metricas_df['distancia_euclidiana'] = (dist_euclidean_norm / dist_euclidean_norm.max()) * 100
        
        # Normalizar distancias mahalanobis (0-100)
        dist_mahalanobis_norm = pd.Series(dist_mahalanobis, index=df.index)
        metricas_df['distancia_mahalanobis'] = (dist_mahalanobis_norm / dist_mahalanobis_norm.max()) * 100
        
        # Normalizar scores isolation forest (-1 a 1 → 0 a 100)
        scores_isolation_norm = pd.Series(scores_isolation, index=df.index)
        metricas_df['score_isolation'] = ((scores_isolation_norm - scores_isolation_norm.min()) / 
                                        (scores_isolation_norm.max() - scores_isolation_norm.min())) * 100
        
        # ========================================
        # 2. MÉTRICA DE EFICIENCIA ANÓMALA
        # ========================================
        
        # Eficiencia esperada vs real
        eficiencia_esperada = self._calcular_eficiencia_esperada(df, irradiance_col, temp_col)
        eficiencia_real = df['ac_power'] / (df[irradiance_col] + 1e-6)  # Evitar división por 0
        
        # Pérdida de eficiencia (0-100%, donde 100% = pérdida total)
        perdida_eficiencia = np.maximum(0, (eficiencia_esperada - eficiencia_real) / eficiencia_esperada * 100)
        metricas_df['perdida_eficiencia'] = np.clip(perdida_eficiencia, 0, 100)
        
        # ========================================
        # 3. MÉTRICA DE DESVIACIÓN Z-SCORE
        # ========================================
        
        # Z-score para variables clave
        metricas_df['zscore_ac_power'] = np.abs(stats.zscore(df['ac_power']))
        metricas_df['zscore_ac_voltage'] = np.abs(stats.zscore(df['ac_voltage']))
        metricas_df['zscore_eficiencia'] = np.abs(stats.zscore(eficiencia_real))
        
        # ========================================
        # 4. MÉTRICA DE CORRELACIÓN ROTA
        # ========================================
        
        # Correlación local vs global (ventana móvil)
        correlacion_global = df[irradiance_col].corr(df['ac_power'])
        correlacion_local = df[irradiance_col].rolling(window=24).corr(df['ac_power']).fillna(correlacion_global)
        
        # Qué tanto se desvía de la correlación esperada
        metricas_df['correlacion_rota'] = np.abs(correlacion_global - correlacion_local) * 100
        
        # ========================================
        # 5. SCORE COMPUESTO DE SEVERIDAD
        # ========================================
        
        # Combinar múltiples métricas con pesos
        pesos = {
            'distancia_mahalanobis': 0.25,    # Más peso por sensibilidad a correlaciones
            'perdida_eficiencia': 0.30,       # Métrica específica de paneles solares
            'distancia_euclidiana': 0.20,     # Distancia básica
            'score_isolation': 0.15,          # Detección ML
            'correlacion_rota': 0.10          # Correlaciones anómalas
        }
        
        score_severidad = (
            metricas_df['distancia_mahalanobis'] * pesos['distancia_mahalanobis'] +
            metricas_df['perdida_eficiencia'] * pesos['perdida_eficiencia'] +
            metricas_df['distancia_euclidiana'] * pesos['distancia_euclidiana'] +
            metricas_df['score_isolation'] * pesos['score_isolation'] +
            metricas_df['correlacion_rota'] * pesos['correlacion_rota']
        )
        
        metricas_df['severidad_compuesta'] = score_severidad
        
        # ========================================
        # 6. CLASIFICACIÓN DE SEVERIDAD
        # ========================================
        
        def clasificar_severidad(score):
            if score >= 80:
                return "🔴 CRÍTICA"
            elif score >= 60:
                return "🟠 ALTA"
            elif score >= 40:
                return "🟡 MEDIA"
            elif score >= 20:
                return "🟢 BAJA"
            else:
                return "⚪ NORMAL"
        
        metricas_df['clasificacion'] = metricas_df['severidad_compuesta'].apply(clasificar_severidad)
        
        self.metricas_df = metricas_df
        self.metricas_calculadas = True
        
        return metricas_df
    
    def _calcular_eficiencia_esperada(self, df, irradiance_col, temp_col):
        """
        Calcular eficiencia esperada basada en condiciones ambientales
        Modelo simplificado: Eficiencia = f(irradiancia, temperatura)
        """
        # Eficiencia base normalizada por irradiancia
        eficiencia_base = df[irradiance_col] * 0.02  # Factor típico de conversión
        
        # Corrección por temperatura (los paneles son menos eficientes con calor)
        temperatura_optima = 25  # °C
        factor_temperatura = 1 - (np.maximum(0, df[temp_col] - temperatura_optima) * 0.004)
        
        return eficiencia_base * factor_temperatura
    
    def analizar_anomalias_por_severidad(self, anomalias_euclidean, anomalias_mahalanobis, 
                                       anomalias_isolation):
        """
        Analizar las anomalías detectadas por nivel de severidad
        """
        if not self.metricas_calculadas:
            raise ValueError("Primero calcule las métricas con calcular_metricas_severidad()")
        
        print("\n🎯 ANÁLISIS POR SEVERIDAD")
        print("="*40)
        
        # Todas las anomalías únicas
        todas_anomalias = set(anomalias_euclidean) | set(anomalias_mahalanobis) | set(anomalias_isolation)
        
        # Crear reporte de severidad
        reporte_severidad = []
        
        for fecha in todas_anomalias:
            if fecha in self.metricas_df.index:
                fila = self.metricas_df.loc[fecha]
                
                # Determinar qué métodos la detectaron
                detectores = []
                if fecha in anomalias_euclidean:
                    detectores.append("Euclidiana")
                if fecha in anomalias_mahalanobis:
                    detectores.append("Mahalanobis")
                if fecha in anomalias_isolation:
                    detectores.append("Isolation")
                
                reporte_severidad.append({
                    'fecha': fecha,
                    'severidad_score': fila['severidad_compuesta'],
                    'clasificacion': fila['clasificacion'],
                    'perdida_eficiencia': fila['perdida_eficiencia'],
                    'dist_mahalanobis': fila['distancia_mahalanobis'],
                    'detectores': ', '.join(detectores),
                    'num_detectores': len(detectores)
                })
        
        # Convertir a DataFrame y ordenar por severidad
        reporte_df = pd.DataFrame(reporte_severidad)
        reporte_df = reporte_df.sort_values('severidad_score', ascending=False)
        
        # Mostrar top 10 más severas
        print("\n🔥 TOP 10 ANOMALÍAS MÁS SEVERAS:")
        print("-" * 80)
        print(f"{'Fecha':<20} {'Severidad':<12} {'Clase':<15} {'Pérdida %':<10} {'Detectores'}")
        print("-" * 80)
        
        for _, row in reporte_df.head(10).iterrows():
            fecha_str = row['fecha'].strftime('%Y-%m-%d %H:%M')
            print(f"{fecha_str:<20} {row['severidad_score']:<11.1f} {row['clasificacion']:<15} "
                  f"{row['perdida_eficiencia']:<9.1f}% {row['detectores']}")
        
        # Estadísticas por clasificación
        print(f"\n📊 DISTRIBUCIÓN POR SEVERIDAD:")
        clasificaciones = reporte_df['clasificacion'].value_counts()
        for clase, count in clasificaciones.items():
            print(f"   {clase}: {count} anomalías")
        
        return reporte_df
    
    def plot_severidad_timeline(self, df, reporte_df):
        """
        Graficar timeline de severidad de anomalías
        """
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
        
        # Gráfica 1: AC Power con anomalías coloreadas por severidad
        ax1.plot(df.index, df['ac_power'], alpha=0.6, color='lightblue', label='AC Power Normal')
        
        # Colorear anomalías por severidad
        colores_severidad = {
            '🔴 CRÍTICA': 'red',
            '🟠 ALTA': 'orange', 
            '🟡 MEDIA': 'yellow',
            '🟢 BAJA': 'green'
        }
        
        for clase, color in colores_severidad.items():
            anomalias_clase = reporte_df[reporte_df['clasificacion'] == clase]
            if len(anomalias_clase) > 0:
                fechas = anomalias_clase['fecha']
                valores = df.loc[fechas, 'ac_power'] if len(fechas) > 0 else []
                ax1.scatter(fechas, valores, c=color, s=50, alpha=0.8, label=clase)
        
        ax1.set_ylabel('AC Power (W)')
        ax1.set_title('Timeline de Anomalías por Severidad')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        
        # Gráfica 2: Distribución de scores de severidad
        ax2.hist(reporte_df['severidad_score'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        ax2.axvline(reporte_df['severidad_score'].mean(), color='red', linestyle='--', 
                   label=f'Media: {reporte_df["severidad_score"].mean():.1f}')
        ax2.set_xlabel('Score de Severidad')
        ax2.set_ylabel('Frecuencia')
        ax2.set_title('Distribución de Scores de Severidad')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

In [ ]:



def aplicar_metricas_severidad(df_hourly, anomalias_euclidean, anomalias_mahalanobis, 
                              anomalias_isolation, dist_euclidean, dist_mahalanobis, 
                              scores_isolation):
    """
    Función principal para aplicar todas las métricas de severidad
    """
    # Crear instancia del analizador
    analizador = SeveridadAnomalias()
    
    # Calcular métricas
    metricas_df = analizador.calcular_metricas_severidad(
        df_hourly, anomalias_euclidean, anomalias_mahalanobis, 
        anomalias_isolation, dist_euclidean, dist_mahalanobis, scores_isolation
    )
    
    # Analizar por severidad
    reporte_severidad = analizador.analizar_anomalias_por_severidad(
        anomalias_euclidean, anomalias_mahalanobis, anomalias_isolation
    )
    
    # Graficar timeline
    analizador.plot_severidad_timeline(df_hourly, reporte_severidad)
    
    return metricas_df, reporte_severidad, analizador

# ==========================================
# 8. FUNCIÓN PRINCIPAL MEJORADA
# ==========================================

def main():
    """
    Función principal que ejecuta todo el análisis con visualizaciones mejoradas
    """
    print("🔋 ANÁLISIS DE ANOMALÍAS EN PANELES FOTOVOLTAICOS")
    print("="*65)
    
    # 1. Cargar datasets
    environment_data, irradiance_data, electrical_data = cargar_datasets()
    
    if any(data is None for data in [environment_data, irradiance_data, electrical_data]):
        print("❌ Error: No se pudieron cargar todos los datasets necesarios")
        return
    
    # 2. Combinar datasets
    df_combined = combinar_datasets(environment_data, irradiance_data, electrical_data)
    
    # 3. Crear promedios horarios
    df_hourly = crear_promedios_horarios(df_combined, 
        metodo_agregacion='mean',  # Puedes cambiar a 'median' o 'weighted_mean'
        incluir_estadisticas=True)

    df_hourly.info()
    
    print(f"\n📊 Dataset final para análisis:")
    print(f"   - Filas: {len(df_hourly)}")
    print(f"   - Columnas: {len(df_hourly.columns)}")
    print(f"   - Rango de fechas: {df_hourly.index.min()} a {df_hourly.index.max()}")
    
    # 4. Detectar anomalías con los 3 métodos (parámetros más selectivos)
    anomalias_euclidean, dist_euclidean, umbral_euclidean = detector_euclidean_distance(df_hourly, percentil=99)
    anomalias_mahalanobis, dist_mahalanobis, umbral_mahalanobis = detector_mahalanobis_distance(df_hourly, percentil=99)
    anomalias_isolation, scores_isolation = detector_isolation_forest(df_hourly, contaminacion=0.01)
    
    # 5. Analizar calidad de las anomalías
    analizar_calidad_anomalias(df_hourly, anomalias_euclidean, anomalias_mahalanobis, anomalias_isolation)
    
    # ========================================
    # 6. VISUALIZACIONES MEJORADAS
    # ========================================
    
    print("\n🎨 GENERANDO VISUALIZACIONES MEJORADAS...")
    
    # Gráfica específica de distancia euclidiana con máscara
    plot_euclidean_distance_with_mask(df_hourly, anomalias_euclidean, dist_euclidean, 
                                     umbral_euclidean, variable_principal='ac_power')
    
    # Gráfica comparativa mejorada (incluye distancias euclidianas)
    plot_anomalies_comparison_mejorado(df_hourly, anomalias_euclidean, anomalias_mahalanobis, 
                                      anomalias_isolation, dist_euclidean, dist_mahalanobis, 
                                      scores_isolation, umbral_euclidean, umbral_mahalanobis,
                                      variable_principal='ac_power')
    
    # 7. NUEVO: Calcular métricas de severidad
    print("\n" + "="*60)
    print("🎯 ANÁLISIS DE SEVERIDAD DE ANOMALÍAS")
    print("="*60)
    
    metricas_df, reporte_severidad, analizador = aplicar_metricas_severidad(
        df_hourly, 
        anomalias_euclidean, 
        anomalias_mahalanobis, 
        anomalias_isolation,
        dist_euclidean,
        dist_mahalanobis, 
        scores_isolation
    )
    
    # 8. Generar lista de anomalías
    generar_lista_anomalias(anomalias_euclidean, anomalias_mahalanobis, anomalias_isolation)
    
    return df_combined,df_hourly, anomalias_euclidean, anomalias_mahalanobis, anomalias_isolation, metricas_df, reporte_severidad

# ==========================================
# 9. FUNCIÓN ALTERNATIVA PARA USAR SOLO EUCLIDIANA
# ==========================================

def analisis_solo_euclidiana():
    """
    Función enfocada solo en el análisis de distancia euclidiana con máscara
    """
    print("🔋 ANÁLISIS EUCLIDIANO DE ANOMALÍAS")
    print("="*50)
    
    # Cargar y preparar datos
    environment_data, irradiance_data, electrical_data = cargar_datasets()
    
    if any(data is None for data in [environment_data, irradiance_data, electrical_data]):
        print("❌ Error: No se pudieron cargar todos los datasets necesarios")
        return
    
    df_combined = combinar_datasets(environment_data, irradiance_data, electrical_data)
    df_hourly = crear_promedios_horarios(df_combined)
    
    # Solo detectar anomalías euclidianas
    anomalias_euclidean, dist_euclidean, umbral_euclidean = detector_euclidean_distance(df_hourly, percentil=99)
    
    # Visualización específica
    plot_euclidean_distance_with_mask(df_hourly, anomalias_euclidean, dist_euclidean, 
                                     umbral_euclidean, variable_principal='ac_power')
    
    return df_hourly, anomalias_euclidean, dist_euclidean, umbral_euclidean

# ==========================================
# 10. EJECUCIÓN
# ==========================================

if __name__ == "__main__":
    # Ejecutar análisis completo
    resultado = main()
    
    # O ejecutar solo análisis euclidiano:
    # resultado_euclidiano = analisis_solo_euclidiana()

In [ ]:
from prophet import Prophet
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 1. Prepara tu DataFrame
# 1. Prepara tu DataFrame
df = resultado[0].copy()
df = df[['measured_on', 'ac_power']].dropna()
df = df[df['ac_power'] > 0]  # Solo valores positivos

# Filtro IQR más estricto
Q1 = df['ac_power'].quantile(0.25)
Q3 = df['ac_power'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['ac_power'] >= Q1 - 1.0 * IQR) & (df['ac_power'] <= Q3 + 1.0 * IQR)]

# Filtro por percentiles extremos
lower = df['ac_power'].quantile(0.02)
upper = df['ac_power'].quantile(0.98)
df = df[(df['ac_power'] >= lower) & (df['ac_power'] <= upper)]

# Suavizado más fuerte


# 2. Resampleo horario
df_hourly = df.set_index('measured_on').resample('D').mean().dropna().reset_index()

# ✅ Detección de gaps grandes
df_hourly['gap'] = df_hourly['measured_on'].diff() > pd.Timedelta('1.5H')
print(f"{df_hourly['gap'].sum()} gaps mayores a 1.5 horas")

# 3. Prepara datos para Prophet
df_prophet = df_hourly.rename(columns={'measured_on': 'ds', 'ac_power': 'y'})[['ds', 'y']]
print(df_prophet.head())
# ✅ Filtra valores pequeños para evitar distorsiones en MAPE
df_prophet = df_prophet[df_prophet['y'] > 5]

# 4. Entrenamiento
print("🔮 INICIANDO ANÁLISIS CON PROPHET")
print("=" * 40)
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode='additive'
)
model.fit(df_prophet)

# 5. Predicción a futuro
# Encuentra la última fecha de tu serie
last_date = df_prophet['ds'].max()
# Define la fecha final deseada
end_date = pd.Timestamp('2025-12-31 23:00:00')
# Calcula cuántas horas hay entre ambas fechas
periods = int((end_date - last_date) / pd.Timedelta(hours=1))

# Genera el dataframe futuro hasta 2025
future = model.make_future_dataframe(periods=360)
forecast = model.predict(future)

# ✅ Evitar valores negativos
forecast['yhat'] = forecast['yhat'].clip(lower=0)
forecast['yhat_lower'] = forecast['yhat_lower'].clip(lower=0)
forecast['yhat_upper'] = forecast['yhat_upper'].clip(lower=0)

# 6. Visualización 
fig1 = model.plot(forecast)
plt.title('🔋 Predicción de Producción de Energía (ac_power)')
plt.xlabel('Fecha')
plt.ylabel('ac_power')
plt.show()

fig2 = model.plot_components(forecast)
plt.show()

# 7. Evaluación del modelo
y_true = df_prophet['y'].reset_index(drop=True)
y_pred = forecast['yhat'][:len(y_true)].reset_index(drop=True)

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true.replace(0, np.nan))) * 100

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

smape_val = smape(y_true, y_pred)

# 8. Mostrar métricas
print(f"📏 MAE:  {mae:.2f}")
print(f"📉 RMSE: {rmse:.2f}")
print(f"📊 MAPE: {mape:.2f}%")
print(f"📊 SMAPE: {smape_val:.2f}%")

# 9. Análisis de residuos
residuals = y_true - y_pred
plt.figure(figsize=(12, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('📈 Análisis de Residuos')
plt.xlabel('Predicción')
plt.ylabel('Error (residuo)')
plt.grid(True)
plt.show()